In this Langchain workflow, I loaded the data into a vectordb (FAISS) and created the retrieval and document chains to help LLM(GPT-4o) model to gather the query and provide the response

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [26]:
# Scrapped and load the data from web
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://python.langchain.com/docs/how_to/#chat-models"
loader = WebBaseLoader(page_url)

doc=loader.load()

In [ ]:
# Splitting/Chunking the data and saving to vectordb after converting to vector embeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter 

# Splitting documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
split_doc = text_splitter.split_documents(doc)

# Convert to a list of Document objects
document_list = [Document(chunk.page_content, metadata=chunk.metadata) for chunk in split_doc]

In [42]:
## Created the embeddings from Open Ai embeddings library and used FAISS for vector db
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vector_storedb = FAISS.from_documents(document_list, embeddings)

type(vector_storedb)

langchain_community.vectorstores.faiss.FAISS

In [51]:
# Querying the vectordb
results = vector_storedb.similarity_search(query='Document Loaders are responsible for what?')
results[0].page_content #Most relevent response based on the query

'How to: cache model responses\nHow to: create a custom LLM class\nHow to: stream a response back\nHow to: track token usage\nHow to: work with local models\n\nOutput parsers\u200b\nOutput Parsers are responsible for taking the output of an LLM and parsing into more structured format.\n\nHow to: parse text from message objects\nHow to: use output parsers to parse an LLM response into structured format\nHow to: parse JSON output\nHow to: parse XML output\nHow to: parse YAML output\nHow to: retry when output parsing errors occur\nHow to: try to fix errors in output parsing\nHow to: write a custom output parser class\n\nDocument loaders\u200b\nDocument Loaders are responsible for loading documents from a variety of sources.\n\nHow to: load PDF files\nHow to: load web pages\nHow to: load CSV data\nHow to: load data from a directory\nHow to: load HTML data\nHow to: load JSON data\nHow to: load Markdown data\nHow to: load Microsoft Office data\nHow to: write a custom document loader'

In [ ]:
# Document chain 

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain #Create chains for passing a list of documents to a model
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template(  
"""
Answer the questions based only on the provided context:
<context>
{context}
</context>
"""
)     ## Here we can provide 

document_chain.invoke({
"input":"Document Loaders are responsible for what?",
"context":[Document(page_content = "Document loaders\nDocument Loaders are responsible for loading documents from a variety of sources.")]
}) #Here we passed documents are context for our question

llm = ChatOpenAI(model='gpt-4o')

# The context is provided to the above prompt using create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11fd2ab90>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11fd95d80>, root_client=<openai.OpenAI object at 0x11fd67880>, root_async_client=<openai.AsyncOpenAI object at 0x11fd2b400>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [65]:
#Retrieval chain: 
from langchain.chains import create_retrieval_chain

retrive_db = vector_storedb.as_retriever() 
retrivel_chain = create_retrieval_chain(retrive_db, document_chain) # Here we create a chain between the vector db and the context to the llm
retrivel_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11b051420>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the questions based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | ChatOpenAI(client=<o

In [66]:

response = retrivel_chain.invoke({'input':'Document Loaders are responsible for what?'})


In [67]:
response['answer']

'1. What are Output Parsers responsible for?\n\n   Output Parsers are responsible for taking the output of an LLM and parsing it into a more structured format.\n\n2. How can you parse JSON output according to the context?\n\n   You can parse JSON output by using output parsers designed to handle JSON data.\n\n3. What methods are mentioned for selecting examples?\n\n   Methods for selecting examples mentioned in the context include selecting examples by length, maximal marginal relevance (MMR), n-gram overlap, and similarity.\n\n4. What is the purpose of Document Loaders?\n\n   Document Loaders are responsible for loading documents from a variety of sources.\n\n5. How can LLM responses be cached?\n\n   LLM responses can be cached, as indicated by the context mentioning "How to cache model responses" and "How to cache chat model responses."'